In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import History 
from tensorflow.keras import backend as K
if K.backend() == 'tensorflow':
    import tensorflow as tf

In [2]:
train_path = './seg_train/seg_train/'
test_path = './seg_test/seg_test/'
Labels = ['buildings', 'forest','glacier','mountain','sea','street']
label_dict = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea', 5:'street'}

In [3]:
def get_data(path):
    images = list()
    lbls = list()
    for label in Labels:
        for image in os.listdir(path+label):
            img = cv2.imread(path+label+r'/'+image)
            img = cv2.resize(img,(50,50))
            images.append(img)
            if label == 'buildings':
                lbl = 0
            elif label == 'forest':
                lbl = 1
            elif label == 'glacier':
                lbl = 2
            elif label == 'mountain':
                lbl = 3
            elif label == 'sea':
                lbl = 4
            else:
                lbl = 5
            lbls.append(lbl)
            
    images = np.array(images)/255
    lbls = np.array(lbls)
    
    return shuffle(images, lbls, random_state=42)

In [4]:
train_images, train_labels = get_data(train_path)

In [5]:
test_images, test_labels = get_data(test_path)

In [6]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=train_images.shape[1:]))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6,activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 64)          0

In [7]:
monitor = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [8]:
model_h = model.fit(train_images,train_labels,epochs=50,validation_split=.25, callbacks=[monitor])

Train on 10525 samples, validate on 3509 samples
Epoch 1/50
10525/10525 [==============================] - 11s 1ms/sample - loss: 1.0427 - accuracy: 0.6006 - val_loss: 0.8498 - val_accuracy: 0.6857
Epoch 2/50
10525/10525 [==============================] - 11s 1ms/sample - loss: 0.7394 - accuracy: 0.7293 - val_loss: 0.6485 - val_accuracy: 0.7629
Epoch 3/50
10525/10525 [==============================] - 11s 1ms/sample - loss: 0.6387 - accuracy: 0.7674 - val_loss: 0.7155 - val_accuracy: 0.7350
Epoch 4/50
10525/10525 [==============================] - 11s 1ms/sample - loss: 0.5600 - accuracy: 0.7957 - val_loss: 0.6587 - val_accuracy: 0.7655
Epoch 5/50
10525/10525 [==============================] - 11s 1ms/sample - loss: 0.4987 - accuracy: 0.8186 - val_loss: 0.6172 - val_accuracy: 0.7700
Epoch 6/50
10525/10525 [==============================] - 11s 1ms/sample - loss: 0.4538 - accuracy: 0.8360 - val_loss: 0.5711 - val_accuracy: 0.7925
Epoch 7/50
10525/10525 [==============================] -

In [9]:
model.evaluate(test_images,test_labels, verbose=1)

3000/3000 [==============================] - 1s 268us/sample - loss: 0.5298 - accuracy: 0.8143


[0.5298499041398367, 0.8143333]